# Previous imports

In [4]:
!pip install 'qiskit[all]'
!pip install qiskit_ibm_runtime
!pip install qiskit_algorithms
!pip install qiskit-machine-learning

  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl size=471384 sha256=5abbdac2b507d09b21c023a2a5525ab2da59db32deb990f25a8f0b27bf0a1e7e
  Stored in directory: /root/.cache/pip/wheels/73/c8/f7/c25448dab74c3acf4848bc25d513c736bb93910277e1528ef4
Successfully built fastdtw


In [5]:
from qiskit.utils import algorithm_globals
from qiskit_machine_learning.datasets import ad_hoc_data
import matplotlib.pyplot as plt
import numpy as np
from qiskit.circuit.library import ZZFeatureMap
import pandas as pd
# from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from sklearn.svm import SVC
from qiskit_machine_learning.algorithms import QSVC

In [6]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

# Dataset creation

In [33]:

# Load the CSV file
data = pd.read_csv("/content/acetylcholinesterase_06_bioactivity_data_3class_pIC50_pubchem_fp.csv")

# Display the first few rows of the loaded data
data.head()


,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,pIC50
0,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6.124938
1,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6.999996
2,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,4.301030
3,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6.522877
4,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,6.096909


In [34]:
import pandas as pd

# Define the number of rows to drop randomly
num_rows_to_drop = 4500
# Randomly select rows to drop
rows_to_drop = data.sample(n=num_rows_to_drop, random_state=42)

# Drop the selected rows from the original DataFrame
data = data.drop(rows_to_drop.index)

# Reset the index if needed
data = data.reset_index(drop=True)

In [35]:
X = data.drop('pIC50', axis=1)
X

,PubchemFP0,PubchemFP1,PubchemFP2,PubchemFP3,PubchemFP4,PubchemFP5,PubchemFP6,PubchemFP7,PubchemFP8,PubchemFP9,...,PubchemFP871,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880
0,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
116,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
117,1,1,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
118,1,1,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [36]:
Y = data.pIC50
Y

0      3.000000
1      3.148742
2      7.221842
3      7.376740
4      5.000000
         ...   
115    6.619787
116    4.898941
117    4.856985
118    6.181773
119    6.638270
Name: pIC50, Length: 120, dtype: float64

In [37]:
print(X.shape)
print(Y.shape)

(120, 881)
(120,)


In [38]:
selection = VarianceThreshold(threshold=(.8 * (1 - .8)))
X = selection.fit_transform(X)

In [39]:
X.shape

(120, 145)

In [40]:
pca = PCA(n_components=10)
X_reduced = pca.fit_transform(X)

In [41]:
X_reduced.shape

(120, 10)

In [42]:
scaler = MinMaxScaler()

# Fit and transform the features using the scaler
X_scaled = scaler.fit_transform(X_reduced)

In [43]:
train_features,test_features, train_labels , test_labels = train_test_split(X_scaled, Y, test_size=0.2)

In [24]:
from qiskit_ibm_runtime import QiskitRuntimeService,Sampler

tokens=""
ibm_quantum_service = QiskitRuntimeService(channel="ibm_quantum", token=tokens)
QiskitRuntimeService.save_account(channel="ibm_quantum", token=tokens,overwrite=True)

In [44]:
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("simulator_statevector")

In [49]:
from qiskit.compiler import transpile
from qiskit.quantum_info import Operator

transpiled_circuit= ZZFeatureMap(feature_dimension=10, reps=1, entanglement="linear")

adhoc_feature_map = transpile(transpiled_circuit,backend = backend, optimization_level = 0)

sampler = Sampler(backend=backend)

fidelity = ComputeUncompute(sampler=sampler)

adhoc_kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=adhoc_feature_map)

In [50]:
print(train_features.shape)
print(test_features.shape)

(96, 10)
(24, 10)


In [51]:
from qiskit_machine_learning.algorithms import QSVR

In [52]:
qsvc = QSVR(quantum_kernel=adhoc_kernel)
qsvc.fit(train_features, train_labels)

qsvc_predictions = qsvc.predict(test_features)

qsvc_score = qsvc.score(test_features, test_labels)
print(f"QSVC classification test score: {qsvc_score}")

ERROR:websocket:ping/pong timed out - goodbye


QSVC classification test score: 0.24827335284807406
